In [ ]:
# !pip install matplotlib tqdm numpy

In [19]:
import argparse
from pathlib import Path
import multiprocessing
from time import time
from collections import defaultdict

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
from multiprocessor.scheduler import MultiprocessorSchedulerType

# self-written modules
from utils.metrics import MultiprocessorFeasibility, calculate_success_rate
from utils.plotters import plot_average_execution_time
from utils.parse import parse_task_file
from multiprocessor.feasibility.review import review_task_sets_in_parallel_multi

In [20]:
# calculate feasibility ratio by parameter
def calculate_feasibility_ratio(schedule_stats):
    feasible = schedule_stats.get(MultiprocessorFeasibility.FEASIBLE_SHORTCUT, 0) + \
               schedule_stats.get(MultiprocessorFeasibility.FEASIBLE_SIMULATION, 0)
    total = feasible + schedule_stats.get(MultiprocessorFeasibility.NOT_SCHEDULABLE_BY_A_SHORTCUT, 0) + \
            schedule_stats.get(MultiprocessorFeasibility.NOT_SCHEDULABLE_BY_A_SIMULATION, 0)
    if total == 0:
        return 0
    return feasible / total * 1.0

In [21]:
# process task sets for Plot 1 and 2 (by number of tasks)
def average_execution_time(config, folder_name):
    """
    Process task sets organized by number of tasks (like in 4-tasks, 10-tasks).
    """
    subfolders = sorted([f for f in Path(folder_name).iterdir() if f.is_dir()], key=lambda x: int(x.name))
    execution_times = {config["alg"]: {}}

    workers = range(1, config["number_of_workers"] + 1)

    for worker_quantity in workers:
        for subfolder in subfolders:
            print(f"Subfolder: {subfolder}")
            num_tasks = 100
            print(f"\nProcessing {num_tasks} tasks in {subfolder}")
            print(f"Evaluating algorithm: {config["alg"].name}")
            schedule_stats = review_task_sets_in_parallel_multi(algorithm=config["alg"],
                                                            folder_name=subfolder,
                                                            num_processors=config["processors"],
                                                            num_clusters=config["clusters"],
                                                            heuristic=config["heuristic"],
                                                            number_of_workers=worker_quantity)
            success_rate = calculate_success_rate(schedule_stats) #TODO execution time
            execution_times[num_tasks] = success_rate

    return execution_times

In [ ]:
config_to_evaluate = {
        "alg": MultiprocessorSchedulerType.PARTITIONED_EDF,
        "heuristic": 'bf',
        "sorting": 'du',
        "processors": 8,
        "clusters": 4,
        "number_of_workers": 32
    }
# python3 main.py tasksets-multiprocessor 8 partitioned -H bf -s du

results = average_execution_time(config=config_to_evaluate, folder_name="tasksets-multiprocessor")


Subfolder: tasksets-multiprocessor/1

Processing 100 tasks in tasksets-multiprocessor/1
Evaluating algorithm: PARTITIONED_EDF


Error processing task set tasksets-multiprocessor/1/taskset-46: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-91: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-85: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-2: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-22: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-50: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-48: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-21: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/1/taskset-96: s

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


Error processing task set tasksets-multiprocessor/3/taskset-235: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/taskset-214: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/taskset-226: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/taskset-208: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/taskset-234: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/taskset-271: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/taskset-285: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/taskset-204: str.partition() takes exactly one argument (2 given)
Error processing task set tasksets-multiprocessor/3/task

In [ ]:
# plot_average_execution_time()